# Let's clean our data.
Our data is in a ```data.json``` file, so we need to fetch the data using the ```read_json()``` function from ```pandas``` to transform it into a dataframe for cleaning.


In [10]:
import json
import pandas as pd

In [11]:
df = pd.read_json('data.json')
df.head(10)

,titles,locations,prices
0,"[3 Bed Townhouse with En Suite at Mtwapa, 4 Be...","[mtwapa, Mombasa CBD, Mombasa Island, Ruiru, K...","[9,000,000, 9,000,000, 9,000,000, 17,500,000, ..."


**There is a problem**

Our df returns a list of titles, locations and prices instead of a singular item.
This is becuase the titles, locations and prices list are defined inside a dictionary in ```scrape.py``` by:
```python

properties = []
titles = []
locations = []
prices = []

# other code

properties.append(
    {
        "titles": titles,
        "locations": locations,
        "prices": prices
    }
)

```

So I'll need to explode it into individual lists. But I encountered a ValueError that the values of titles, locations and prices do not match, so let's check the length of each list from ```data.json```

In [12]:
with open('data.json', 'r') as f:
    data = json.load(f)

In [13]:
len(data[0]["titles"])

2456

In [14]:
len(data[0]["locations"])

2456

In [15]:
len(data[0]["prices"])

7293

**Problem identified.**

So the problem is, the length of the prices list is greater than both prices and locations.
I'll truncate the values of the ```prices``` column to the 2457th index (to capture the 2456th price) then explode to individual columns

In [18]:
df["prices"] = df["prices"].apply(lambda x: x[:2456])
df = df.explode(["titles", "locations", "prices"]).reset_index(drop=True)
df.head(10)

,titles,locations,prices
0,3 Bed Townhouse with En Suite at Mtwapa,"mtwapa, Mombasa CBD, Mombasa Island","9,000,000"
1,4 Bed Townhouse with En Suite at Ruiru,Ruiru,"9,000,000"
2,5 Bed Townhouse with En Suite in Kitisuru,"Kitisuru, Westlands","9,000,000"
3,4 Bed Townhouse with En Suite at Lavington Green,"Lavington green, Lavington","17,500,000"
4,5 Bed Villa with En Suite in Runda,"Runda, Westlands","17,500,000"
5,5 Bed Townhouse with En Suite in Kitisuru,"Kitisuru, Westlands","17,500,000"
6,5 Bed House with En Suite in Lower Kabete,"Lower Kabete, Westlands","290,000,000"
7,5 Bed House with En Suite in Lavington,Lavington,"290,000,000"
8,5 Bed Townhouse with En Suite at Langata Road-,"1Langata road-Kitengela lane, Langata road-, L...","290,000,000"
9,4 Bed House with En Suite at Ruiru Easterbypass,"Kamakis, Ruiru Easterbypass, Ruiru","62,000,000"


In [20]:
df.loc[9]

titles       4 Bed House with En Suite at Ruiru Easterbypass
locations                 Kamakis, Ruiru Easterbypass, Ruiru
prices                                            62,000,000
Name: 9, dtype: object